**Multiclass Fish Image Classification using Pre-trained models**

**MobileNetV2**

In [ ]:
#import the required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#set the device
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
#transformation for training data
train_transform = transforms.Compose(
    [

        transforms.RandomResizedCrop(224, scale = (0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor()])

In [ ]:
test_val_transform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor()]
)

In [ ]:
# load the datasets
train_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/train', transform = train_transform)
test_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/test', transform = test_val_transform)
val_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/val', transform = test_val_transform)

In [ ]:
# create dataloder

train_data_loader = DataLoader(train_datasets, batch_size = 5, shuffle = True)
test_data_loader = DataLoader(test_datasets, batch_size= 5, shuffle = True)
val_data_loader = DataLoader(val_datasets, batch_size = 5, shuffle= True)


In [ ]:
# load the pre trained model MobileNet V2
model = models.mobilenet_v2(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 173MB/s]


In [ ]:
model.classifier[1] = nn.Linear(model.last_channel, 11)

In [ ]:
model = model.to(device)

In [ ]:
#loss function and optimizer

cl = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr = 0.001)

In [ ]:
# train the model
epochs = 25
for i in range(epochs):
  model.train()
  total_loss = 0.0
  total = 0
  correct = 0

  for img,tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)

    optimizer.zero_grad()
    outputs = model(img)
    loss = cl(outputs, tar)
    loss.backward()
    optimizer.step()


    total_loss += loss.item()
    _, pred = torch.max(outputs, 1)
    correct += (pred == tar).sum().item()
    total += tar.size(0)


  print(f"Epochs [{i + 1}/{epochs}], loss {total_loss:.4f}")

Epochs [1/25], loss 370.2602
Epochs [2/25], loss 410.4366
Epochs [3/25], loss 371.0588
Epochs [4/25], loss 374.4433
Epochs [5/25], loss 377.6933
Epochs [6/25], loss 378.3074
Epochs [7/25], loss 378.7689
Epochs [8/25], loss 378.0514
Epochs [9/25], loss 389.6587
Epochs [10/25], loss 385.8933
Epochs [11/25], loss 368.1598
Epochs [12/25], loss 426.0201
Epochs [13/25], loss 368.6740
Epochs [14/25], loss 350.4293
Epochs [15/25], loss 368.9431
Epochs [16/25], loss 402.4252
Epochs [17/25], loss 413.1719
Epochs [18/25], loss 345.7085
Epochs [19/25], loss 369.6681
Epochs [20/25], loss 378.1057
Epochs [21/25], loss 393.7274
Epochs [22/25], loss 350.4544
Epochs [23/25], loss 394.5706
Epochs [24/25], loss 380.4732
Epochs [25/25], loss 429.0091


In [ ]:
#Evaluate validation data

model.eval()
all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in val_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)


    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())



  #accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"val_accuracy: {accuracy* 100 :.2f}%")

  #precision
  precision = precision_score(all_tars , all_preds, average = 'macro')
  print(f"val_precision: {precision * 100 :.2f}%")

  # recall
  recall = recall_score(all_tars , all_preds, average = 'macro')
  print(f"val_recall: {recall *100 :.2f}%")

  #f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"val_f1_score: {f1score *100:.2f}%")

  # Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

val_accuracy: 98.90%
val_precision: 97.51%
val_recall: 98.15%
val_f1_score: 97.80%
confusion matrix
[[185   2   0   0   0   0   0   0   0   0   0]
 [  1   9   0   0   0   0   0   0   0   0   0]
 [  0   0 105   0   0   0   0   0   0   0   0]
 [  0   0   0  90   1   0   3   0   0   0   0]
 [  0   0   0   0  97   0   0   0   0   0   0]
 [  0   0   0   0   0  90   0   0   0   0   0]
 [  0   0   0   0   0   0 113   0   0   0   0]
 [  0   0   1   1   0   0   0  95   0   0   0]
 [  0   0   0   0   0   0   0   0 100   0   0]
 [  0   0   1   0   1   1   0   0   0  98   0]
 [  0   0   0   0   0   0   0   0   0   0  98]]


In [ ]:
# Evaluation on test data

model.eval()

all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in test_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


# accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"Test Accuracy: {accuracy *100:.2f}%")

# precision
  precision = precision_score(all_tars, all_preds, average = 'macro')
  print(f"Test precision: {precision *100:.2f}%")

#recall
  recall = recall_score(all_tars, all_preds, average = 'macro')
  print(f"Test recall: {recall *100:.2f}%")

#f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"Test f1_score: {f1score *100:.2f}%")

# Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

Test Accuracy: 98.78%
Test precision: 96.53%
Test recall: 97.57%
Test f1_score: 97.00%
confusion matrix
[[515   4   0   0   1   0   0   0   0   0   0]
 [  2  11   0   0   0   0   0   0   0   0   0]
 [  0   0 298   0   0   0   0   0   0   0   0]
 [  0   0   0 295   0   0  10   0   0   0   0]
 [  0   0   0   0 286   0   0   0   0   0   0]
 [  0   0   0   0   0 287   0   0   0   4   0]
 [  0   0   0   0   0   0 273   0   0   0   0]
 [  0   0   6   4   0   0   0 316   0   1   0]
 [  0   0   0   0   0   0   0   0 289   0   0]
 [  0   0   0   0   1   1   0   0   0 291   0]
 [  0   0   0   1   0   0   0   4   0   0 287]]


In [ ]:
# evaluate the train data

model.eval()

all_preds = []
all_tars = []


with torch.no_grad():
  for img, tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _,preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


# accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"Train Accuracy: {accuracy* 100:.2f}%")

#precision
  precision = precision_score(all_tars, all_preds, average = 'macro')
  print(f"Train precision: {precision*100:.2f}%")

#recall
  recall = recall_score(all_tars, all_preds, average = 'macro')
  print(f"Train recall:{recall*100:.2f}%")


#f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"Train f1_score: {f1score *100:.2f}%")

# Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

Train Accuracy: 99.21%
Train precision: 96.95%
Train recall:99.33%
val f1_score: 97.95%
confusion matrix
[[1081   11    0    2    0    1    1    0    0    0    0]
 [   0   30    0    0    0    0    0    0    0    0    0]
 [   0    0  568    0    1    0    0    0    0    0    0]
 [   0    0    0  559    3    0    3    1    0    0    0]
 [   0    0    0    0  573    0    0    0    0    0    0]
 [   0    0    0    0    0  578    0    0    0    1    0]
 [   0    0    0    1    0    0  570    0    0    0    0]
 [   0    0    5    1    0    0    0  532    0    0    0]
 [   0    0    0    0    0    0    0    0  575    1    0]
 [   0    0    1    0    3    1    0    0    0  541    1]
 [   0    0    0    3    0    0    1    7    0    0  569]]


In [ ]:
# save the model

torch.save(model.state_dict(), 'mobilenetv2_fish_model.pth')